In [2]:
import splitfolders

input_folder = '/Users/harshilpatel/Desktop/plant recognition/plantrecognition 2/New_data'
output_folder = '/Users/harshilpatel/Desktop/plant recognition/plantrecognition 2/working'

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.7, .15, .15))

Copying files: 0 files [00:00, ? files/s]

Copying files: 3588 files [00:01, 2498.38 files/s]


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/Users/harshilpatel/Desktop/plantrecognition/working/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    '/Users/harshilpatel/Desktop/plantrecognition/working/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    '/Users/harshilpatel/Desktop/plantrecognition/working/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/harshilpatel/Desktop/plantrecognition/working/train'

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = base_model = MobileNetV2(weights='/Users/harshilpatel/Downloads/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

import json

# Save class indices to a JSON file
class_indices_path = '/Users/harshilpatel/Desktop/plantrecognition/model/class_indices.json'
with open(class_indices_path, 'w') as json_file:
    json.dump(train_generator.class_indices, json_file)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

model.save('/Users/harshilpatel/Desktop/plantrecognition/model/plant-recognition-model.h5')


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 236ms/step - accuracy: 0.9886 - loss: 0.0325 - val_accuracy: 0.9869 - val_loss: 0.0823
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 236ms/step - accuracy: 0.9918 - loss: 0.0343 - val_accuracy: 0.9813 - val_loss: 0.0675
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 236ms/step - accuracy: 0.9876 - loss: 0.0425 - val_accuracy: 0.9757 - val_loss: 0.0668
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 238ms/step - accuracy: 0.9916 - loss: 0.0285 - val_accuracy: 0.9851 - val_loss: 0.0455
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 238ms/step - accuracy: 0.9877 - loss: 0.0410 - val_accuracy: 0.9851 - val_loss: 0.0578
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 238ms/step - accuracy: 0.9847 - loss: 0.0466 - val_accuracy: 0.9888 - val_loss: 0.0527
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 19s 235ms/step - accuracy: 0.9914 - loss: 0.0248 - val_accuracy: 0.9944 - val_loss: 0.0356
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 233ms/step - accuracy: 0.9917 - loss: 0.0225 - val_accu

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc*100}%")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 190ms/step - accuracy: 0.1321 - loss: 2.2145
Test accuracy: 11.743119359016418%
